## Desenvolvendo um classificador capaz de analisar se um comentário traz uma mensagem positiva ou negativa

link:https://www.kaggle.com/luisfredgs/imdb-ptbr?select=imdb-reviews-pt-br.csv

In [1]:
import pandas as pd

In [2]:
dados = pd.read_csv('imdb-reviews-pt-br.csv')

In [3]:
dados.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


- Criando uma coluna numérica para representar a coluna **sentiment**

In [27]:
dados.sentiment.value_counts(normalize=True)

neg    0.500718
pos    0.499282
Name: sentiment, dtype: float64

In [28]:
dados['classificacao'] = dados['sentiment'].apply(lambda x: 1 if x=='pos' else 0)

In [29]:
dados.head()

,id,text_en,text_pt,sentiment,classificacao
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,0
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,0
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,0
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,0
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,0


## Vetorizando um texto

- Bag of words
- Criando uma matriz representativa com as palavras.

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

In [33]:
texto = ['Assisti um ótimo filme','Vi a um filme muito ruim']


vetorizar = CountVectorizer(lowercase=False)
bag_of_words = vetorizar.fit_transform(texto)

In [36]:
vetorizar.get_feature_names()

['Assisti', 'Vi', 'filme', 'muito', 'ruim', 'um', 'ótimo']

In [42]:
# criando umdataframe com as palavras

matriz_esparsa = pd.DataFrame.sparse.from_spmatrix(bag_of_words,columns=vetorizar.get_feature_names())

In [43]:
matriz_esparsa

,Assisti,Vi,filme,muito,ruim,um,ótimo
0,1,0,1,0,0,1,1
1,0,1,1,1,1,1,0


In [44]:
print(bag_of_words.shape)

(2, 7)


In [46]:
#  criando a matriz esparsa com os dados do dataset
vetorizar = CountVectorizer(lowercase=False, max_features=50)
bag_of_words = vetorizar.fit_transform(dados.text_pt)
print(bag_of_words.shape)

(49459, 50)


## Aplicando  Machine learning para classificação de textos

- Divisão em treino e teste
- Treinando um modelo

In [6]:
from sklearn.model_selection import train_test_split

In [55]:

X_treino, X_teste, y_treino, y_teste = train_test_split(bag_of_words,dados.classificacao, random_state=42, test_size=0.3)

In [52]:
from sklearn.linear_model import LogisticRegression

In [60]:
lr = LogisticRegression(solver='lbfgs')
lr.fit(X_treino,y_treino)
acuracia = lr.score(X_teste,y_teste)

print(f' A acurácia foi de {acuracia}')

 A acurácia foi de 0.6582423507211215


In [63]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [64]:
%matplotlib inline
from wordcloud import WordCloud

Analisnado com **textblob**

In [12]:
from textblob import TextBlob

In [14]:
def sentimento(frase):
    tb = TextBlob(frase)
    tb_en = tb.translate(to='en')
    polaridade = tb_en.sentiment.polarity
    return polaridade

In [23]:
# dados['analise_sentimentos'] = dados.text_pt.apply(sentimento)

In [18]:
dados.text_pt[:10].apply(sentimento)

0    0.023333
1   -0.002582
2   -0.147093
3    0.070000
4    0.100450
5   -0.058869
6   -0.155417
7   -0.045635
8   -0.008333
9    0.111514
Name: text_pt, dtype: float64

In [25]:
dados.text_pt[9]

'Fazendeiros ricos em Buenos Aires têm uma longa política de não-comércio com os Crawford de Manhattan, mas o que acontece quando o filho latino de bigode cai por um certo Crawford com olhos brilhantes, cabelos loiros e alguns movimentos alegres na pista de dança? O musical da 20th Century-Fox tem um verniz brilhante, mas parece um pouco ultrapassado nas bordas. É muito pesado na dança frenética, como ginástica, extremamente fina na história. Betty Grable, uma substituta de décima primeira hora de Alice Faye, dá um impulso, apesar de ela estar emparelhada com Don Ameche, de pele castanha e magricela. Também é bom: Charlotte Greenwood como Bettys tia, uma motorista de limusine que constantemente adormece no trabalho, e Carmen Miranda interpretando quem mais ?. As filmagens de ações da Argentina superam de longe a ação filmada no backlot da Fox, e algumas das performances de apoio são bastante terríveis. Até o final do grande final de corrida de cavalos, a maioria dos espectadores terá o